# Trabajo Práctico

Elegir un tema, persona, situación o evento y crear un sistema que:
- Obtenga los Tweets asociados con dicho tema (Hashtag y/o Palabras claves)
- Aplique algunas de las técnicas de PNL (Tokenización, POS, Lematización, Derivación, Polarización)
- Obtener algunos de los siguientes datos (¿Quíenes están involucrados?, ¿De qué se trata?, ¿En qué lugares ocurre la acción?, ¿Qué opinion o sentimientos expresa la gente?
- Generar resultados (Gráficos, tablas )

## Obtener datos

In [18]:
import os
import pandas as pd
import requests
from dotenv import load_dotenv

# Cargar valores del archivo .env en las variables de entorno
load_dotenv()
# Cargar valor del token a variable
bearer_token = os.environ.get("BEARER_TOKEN")

url = "https://api.twitter.com/2/tweets/search/recent"
headers = {
    "Authorization": f"Bearer {bearer_token}",
    "User-Agent":"v2FullArchiveSearchPython"
} 
params = {
    'query': f'#metaverse lang:en -is:retweet',
    'expansions': 'author_id',
    'max_results':50
}
# Función para obtener hasta 10 paginas del tema buscado.
def get_data(url,params):
    results = []

    for _ in range(5):
        response = requests.get(url, headers=headers, params=params)
        # Generar excepción si la respuesta no es exitosa
        if response.status_code != 200:
            raise Exception(response.status_code, response.text)
        data = response.json()['data']
        meta_data = dict(response.json())['meta']
        results.append(pd.json_normalize(data))
        if 'next_token' not in meta_data:
            break
        else:
            token = meta_data['next_token']
            params = {
                'query': f'#metaverse lang:en -is:retweet',
                'expansions': 'author_id',
                'next_token':token,
                'max_results':50
            }
    
    return pd.concat(results)

df = get_data(url,params)
df.to_csv('metaverse.csv')  

In [19]:
df


,author_id,id,text
0,1389675288255205378,1454461866906894339,bull $TPT price is cheap. buy cheap.\nSign up ...
1,1448338535342231553,1454461853929639943,@CryptoGodJohn I'm definitely not underexposed...
2,1441702148257955843,1454461843993292804,ILOVE #Elemongame Partner #Corsair\n#GameNft #...
3,1440648481970950152,1454461830772899843,✅10 members will get $50 #SAPCE token randomly...
4,44817501,1454461819372724238,@BarrySilbert And if you wanna know abt #metav...
...,...,...,...
45,1344950064859398150,1454459422021861387,@MonsterArenaBSC It is really a legit project....
46,1345616792694112257,1454459414425923584,@GenxAnalytics @DeathRoad_io @DecentralGames @...
47,1386250183416029184,1454459381144166405,@GunHunterNFT Specially Thanks sir\nFor sharin...
48,1392591321840373764,1454459370037784577,#HappyHalloween #halloween2021 #SolanaAirdrop...
